In [ ]:
import piplite
await piplite.install('shapely')
await piplite.install('pypi/fast_histogram-0.14-cp39-abi3-pyodide_2024_0_wasm32.whl')
await piplite.install('pypi/mpl_scatter_density-0.8-py3-none-any.whl')
await piplite.install('pypi/glfw-2.9.0-py3-none-any.whl')
await piplite.install('pypi/vispy-0.15.2-cp312-cp312-pyodide_2024_0_wasm32.whl')

# Do things like this to get around the fact that pyzmq is missing
await piplite.install("glue-core")
await piplite.install("bqplot")
await piplite.install("bqplot_image_gl")
await piplite.install("ipykernel")
await piplite.install("ipyvue")
await piplite.install("ipyvuetify")
await piplite.install("ipywidgets")
await piplite.install("pypi/glue_jupyter-0.10.2.dev520+gafac49aed-py3-none-any.whl", deps=False)

from glue.core import Data
from glue_jupyter import JupyterApplication
from glue_jupyter.bqplot.histogram import BqplotHistogramView
from glue_jupyter.bqplot.scatter import BqplotScatterView
from ipyvuetify import VuetifyTemplate
from ipywidgets import DOMWidget, widget_serialization
from traitlets import Instance

from random import randint


class App(VuetifyTemplate):

    template = """
        <template>
          <v-app>
            <v-row>
              <jupyter-widget :widget="scatter"/>
            </v-row>
            <v-row>
              <jupyter-widget :widget="histogram"/>
            </v-row>
          </v-app>
        </template>
    """

    histogram = Instance(DOMWidget, allow_none=True).tag(sync=True, **widget_serialization)
    scatter = Instance(DOMWidget, allow_none=True).tag(sync=True, **widget_serialization)

    def __init__(self):
        self.app = JupyterApplication()

        N = 500
        M = 50
        x = [randint(0, M) for _ in range(N)]
        y = [randint(0, M) for _ in range(N)]
        z = [randint(0, M) for _ in range(N)]
        self.glue_data = Data(label="Data", x=x, y=y, z=z)
        self.app.data_collection.append(self.glue_data)

        hv = self.app.new_data_viewer(BqplotHistogramView, data=self.glue_data)
        self.histogram = hv.figure
        sv = self.app.new_data_viewer(BqplotScatterView, data=self.glue_data)
        self.scatter = sv.figure

        super().__init__()

App()